In [8]:
import pandas as pd
import argparse
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import math
import random 
from sklearn.linear_model import Ridge, Lasso

In [9]:
# Load dataset
df = pd.read_csv("space_mountain_with_holiday_weather_lag_new.csv")

# One-hot encode categorical features
df = pd.get_dummies(df, columns=['Day of Week']) 

# ------ SPLITTING DATASET ------
train_years = ['14', '15', '16', '17', '18', '19', '22']
dev_years = ['23']
test_years = ['24', '25']

df['Year'] = df['Date'].apply(lambda x: x.split('/')[-1])  # Extract year as a string
df = df[df['Time of Day'] >= 450]
df['Time_sq'] = df['Time of Day'] ** 2
df['Time_sin'] = np.sin(2 * np.pi * df['Time of Day'] / 1440)
df['Time_cos'] = np.cos(2 * np.pi * df['Time of Day'] / 1440)

train_df = df[df['Year'].isin(train_years)]
dev_df = df[df['Year'].isin(dev_years)]
test_df = df[df['Year'].isin(test_years)]

print("Train size:", train_df.shape)
print("Dev size:", dev_df.shape)
print("Test size:", test_df.shape)

# getting datasets
def get_X_y(dataframe):
    y = dataframe['Wait Time'].values
    X = dataframe.drop(columns=['Wait Time', 'Date', 'Time', 'Year'])
    return X, y

X_train, y_train = get_X_y(train_df)
X_dev, y_dev = get_X_y(dev_df)
X_test, y_test = get_X_y(test_df)

Train size: (30844, 26)
Dev size: (5056, 26)
Test size: (7142, 26)


In [10]:
ridge = Ridge(alpha=1.0)
lasso = Lasso(alpha=1.0, max_iter=10000)

ridge.fit(X_train, y_train)
lasso.fit(X_train, y_train)

Lasso(max_iter=10000)

In [11]:
# Define alpha values to try
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

best_ridge_rmse = float('inf')
best_lasso_rmse = float('inf')
best_ridge_alpha = None
best_lasso_alpha = None

# Grid search over alphas using dev set
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    lasso = Lasso(alpha=alpha, max_iter=10000)

    ridge.fit(X_train, y_train)
    lasso.fit(X_train, y_train)

    ridge_dev_pred = ridge.predict(X_dev)
    lasso_dev_pred = lasso.predict(X_dev)

    ridge_rmse = np.sqrt(mean_squared_error(y_dev, ridge_dev_pred))
    lasso_rmse = np.sqrt(mean_squared_error(y_dev, lasso_dev_pred))

    print(f"Alpha: {alpha:.2f} | Ridge Dev RMSE: {ridge_rmse:.2f} | Lasso Dev RMSE: {lasso_rmse:.2f}")

    if ridge_rmse < best_ridge_rmse:
        best_ridge_rmse = ridge_rmse
        best_ridge_alpha = alpha

    if lasso_rmse < best_lasso_rmse:
        best_lasso_rmse = lasso_rmse
        best_lasso_alpha = alpha

/Users/jeremypogue/miniforge3/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=8.80359e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/jeremypogue/miniforge3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.564e+06, tolerance: 1.977e+03
  model = cd_fast.enet_coordinate_descent(
/Users/jeremypogue/miniforge3/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=1.32047e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Alpha: 0.01 | Ridge Dev RMSE: 15.66 | Lasso Dev RMSE: 15.64
Alpha: 0.10 | Ridge Dev RMSE: 15.66 | Lasso Dev RMSE: 15.68
Alpha: 1.00 | Ridge Dev RMSE: 15.66 | Lasso Dev RMSE: 15.71
Alpha: 10.00 | Ridge Dev RMSE: 15.65 | Lasso Dev RMSE: 15.69
Alpha: 100.00 | Ridge Dev RMSE: 15.65 | Lasso Dev RMSE: 16.18


In [12]:
# Train final models with best alpha on train + dev and evaluate on test
X_train_dev = pd.concat([X_train, X_dev])
y_train_dev = np.concatenate([y_train, y_dev])

final_ridge = Ridge(alpha=best_ridge_alpha)
final_lasso = Lasso(alpha=best_lasso_alpha, max_iter=10000)

final_ridge.fit(X_train_dev, y_train_dev)
final_lasso.fit(X_train_dev, y_train_dev)

ridge_test_pred = final_ridge.predict(X_test)
lasso_test_pred = final_lasso.predict(X_test)

ridge_test_rmse = np.sqrt(mean_squared_error(y_test, ridge_test_pred))
lasso_test_rmse = np.sqrt(mean_squared_error(y_test, lasso_test_pred))

print(f"\nBest Ridge Alpha: {best_ridge_alpha} → Test RMSE: {ridge_test_rmse:.2f}")
print(f"Best Lasso Alpha: {best_lasso_alpha} → Test RMSE: {lasso_test_rmse:.2f}")


Best Ridge Alpha: 100.0 → Test RMSE: 15.02
Best Lasso Alpha: 0.01 → Test RMSE: 15.02


/Users/jeremypogue/miniforge3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.181e+06, tolerance: 2.210e+03
  model = cd_fast.enet_coordinate_descent(


In [13]:
lasso_best = Lasso(alpha=1.0, max_iter=10000)
lasso_best.fit(X_train, y_train)

# Display non-zero feature coefficients
coef_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Coefficient": lasso_best.coef_
})
print(coef_df[coef_df["Coefficient"] != 0].sort_values(by="Coefficient", key=abs, ascending=False))


                Feature  Coefficient
10  Wait_Time_30min_Ago     0.435172
11  Wait_Time_60min_Ago     0.169167
6        Lag1_Wait_Time     0.126471
7        Lag2_Wait_Time     0.090412
4           Temperature    -0.062898
1           Time of Day     0.047974
8        Lag3_Wait_Time     0.042498
0                 Month     0.030254
9      RollingMean_Lag3     0.005601
19              Time_sq    -0.000028
